In [ ]:
from torch.nn import LSTM
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn as nn
import pandas as pd
import numpy as np
import random
import linecache
from tqdm import tqdm
import time
import pickle as pkl
import math
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

In [ ]:
root_path = Path(r"../data/")

In [ ]:
def sample_data():
    datapath = root_path/"train_full.csv"
    train_df_iter = pd.read_csv(datapath, chunksize=500000, encoding="gbk")
    rand_list = [random.randint(0, 118042) for i in range(128)]
    counter = 0
    demo_df = pd.DataFrame()
    for train_df in tqdm(train_df_iter):
    #     print(train_df.head())
    #     print(train_df["小区编号"].unique())
        counter += train_df.loc[train_df["小区编号"].isin(rand_list)].shape[0]
        demo_df = pd.concat([demo_df, train_df.loc[train_df["小区编号"].isin(rand_list)]])
    return demo_df

In [ ]:
def data_filter(demo_df):
    demo_df = demo_df.rename(columns={"日期": "Date", "时间":"Time", "小区编号":"ID", "上行业务量GB":"Upload(GB)", "下行业务量GB":"Download(GB)"})
    mapping = {'018-04-01':"2018/4/1", '018-04-02':"2018/4/2", '018-04-03':"2018/4/3", '018-04-04':"2018/4/4", '018-04-08':"2018/4/8", '018-04-09':"2018/4/9", '018-04-10':"2018/4/10"}
    demo_df["Date"] = demo_df.Date.map(lambda x: x if x not in mapping else mapping[x])
    demo_df["DateTime"] = demo_df["Date"] + ' '+demo_df["Time"]
    demo_df["DateTime"] = pd.to_datetime(demo_df["DateTime"])
    demo_df = demo_df.set_index("DateTime")
    return demo_df
# t_demo_df = data_filter(temp_demo_df)
# t_demo_df.head()

In [ ]:
def save_df(df, name):
    df.to_csv(root_path/name)

In [ ]:
def read_test_data(demo=True):
    test_path = root_path/("test_demo.csv" if demo else "train_test.csv")
    test_df = pd.read_csv(test_path, index_col="DateTime")
    return test_df

In [ ]:
def read_train_data(demo=True):
    def gen_iter_df():
        for g in root_path.glob("train_full_split_*.csv"):
            yield pd.read_csv(g, index_col="DateTime")
    if demo:
        demo_path = root_path/("train_demo.csv")
        demo_df = pd.read_csv(demo_path, index_col="DateTime")
    else:
        demo_df = gen_iter_df()
        
    return demo_df
# demo_df = read_train_data()
# demo_df.head()

In [ ]:
def make_data(df):
    """
        能够从df直接出能够用来训练的数据
        输出pad之后的数据和每个数据的长度，以及每个值都属于什么ID
    """
    def padding(data):
        max_length = 0
        lengths = []
        for item in data:
            lengths.append(item.shape[0])
        max_length = max(lengths)
        data_new = np.array([np.concatenate([item, np.zeros((max_length-item.shape[0], item.shape[1]))]) for item in data])
        return data_new, lengths
    df = df.drop(df[(df["Upload(GB)"].isna())|(df["Download(GB)"].isna())].index)
    assert df["Upload(GB)"].isna().sum() == 0, "Upload(GB) has nan"
    assert df["Download(GB)"].isna().sum() == 0, "Download(GB) has nan"
    data_df = df.sort_values(by=["ID", "DateTime"])
    index = []
    data = []
    final_df = pd.DataFrame()
    for item in data_df["ID"].unique():
        item_df = data_df.loc[data_df["ID"] == item]
        index.append(item)
        data.append(item_df.loc[:, ["Upload(GB)", "Download(GB)"]].values)
#         data.append(item_df["Upload(GB)"].tolist())
    #     print(len(item_df["Upload(GB)"].values))
#         data_D.append(item_df["Download(GB)"].tolist())
    #     print(len(item_df["Download(GB)"].values))
    data_pad, data_lengths = padding(data)
    return index, torch.Tensor(data_pad), data_lengths
# index, data_pad, data_lengths = make_data(demo_df)
# data_pad.shape, len(data_lengths), len(index)

In [ ]:
# pkl.dump(["index, data_pad_T, data_lengths", index, data_pad_T, data_lengths], open(r"D:\Dataset\MathorCup\train_data.pkl", "wb"))

In [ ]:
# _, index, data_pad_T, data_lengths = pkl.load(open(r"D:\Dataset\MathorCup\train_data.pkl", "rb"))
# _

# 模型处理

## LSTM模型必须需要三维
- (batch_size, time_sequence, features)

In [ ]:
class MathorCup(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_first=True, dropout=0.5, cuda=False, num_layers=2, cuda_card=0):
        super(MathorCup, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_first = batch_first
        self.dropout = dropout
        self.cuda = cuda
        self.cuda_card = cuda_card
        self.num_layers = num_layers
        
#         self.d_model = 512
        self.device = torch.device("cpu")
        if self.cuda:
            torch.cuda.set_device(self.cuda_card)
#             self.device = torch.device("cuda:{}".format(self.cuda_card))
#         self.linear = nn.Linear(self.input_size, self.d_model)
        self.norm = nn.LayerNorm(self.input_size)
        self.encoder = nn.LSTM(
            input_size=self.input_size, 
            hidden_size=self.hidden_size, 
            batch_first=self.batch_first, 
            dropout=self.dropout, 
            num_layers=self.num_layers
        )
        self.linear_h = nn.Linear(self.hidden_size, self.hidden_size)
        self.linear_c = nn.Linear(self.hidden_size, self.hidden_size)
        self.eval_decoder = nn.LSTMCell(
            input_size=self.input_size, 
            hidden_size=self.hidden_size, 
            bias=True
        )
        self.train_decoder = nn.LSTM(
            input_size=self.input_size, 
            hidden_size=self.hidden_size, 
            batch_first=self.batch_first, 
            dropout=self.dropout, 
            num_layers=self.num_layers
        )
        self.output = nn.Linear(in_features=self.hidden_size, out_features=self.output_size)
        if self.cuda:
            self.norm = self.norm.cuda()
            self.encoder = self.encoder.cuda()
            self.linear_h = self.linear_h.cuda()
            self.linear_c = self.linear_c.cuda()
            self.train_decoder = self.train_decoder.cuda()
            self.eval_decoder = self.eval_decoder.cuda()
            self.output = self.output.cuda()
    
    def encode(self, x):
        if self.cuda:
            x = x.cuda()
        x = self.norm(x)
#         print(x.shape)
#         x = self.linear(x)
#         y = self.linear(y)
#         print(x.shape)
#         x = x.permute(1, 0, 2)
#         y = y.permute(1, 0, 2)
#         out = self.transformer(x, y)
#         out = out.permute(1, 0, 2)
#         print(x.mean())
        o, (h, c) = self.encoder(x)
        
        return o, (h, c)
    
    def attention(self, o, y):
        return torch.zeros_like(y)
    
    def train_decode(self, y, h_c):
        if self.cuda:
            y = y.cuda()
        out, h_c = self.train_decoder(y, h_c)
        outputs = self.output(out)
        return outputs
    
    def evaluate_decode(self, y_t, h_c, num):
        # 这里比纯lstm慢太多了
        if self.cuda:
            y_t = y_t.cuda()
        h, c = h_c
        outputs = []
        for i in range(num):
#             y_t = torch.squeeze(y_t, dim=1)
            h, c = self.linear_h(h), self.linear_c(c)
            h, c = torch.squeeze(h, dim=0), torch.squeeze(c, dim=0)
            
            h, c = self.eval_decoder(y_t, (h, c))
            out = self.output(h)
            y_t = out
            outputs.append(out)
        outputs = torch.stack(outputs, dim=0).permute(1, 0, 2)
        return outputs
        
    def forward(self, x, y):
        self.train()
        o, (h, c) = self.encode(x)
#         att = self.attention(o, y)
        
        out = self.train_decode(y, (h, c))
        return out.cpu()
#         o1 = self.output(x1)
#         return o1.cpu()
    def evaluate(self, x, num):
        self.eval()
        o, (h, c) = self.encode(x)
        out = self.evaluate_decode(x[:, -1, :], (h, c), num)
        return out.cpu()
        
        

In [ ]:
from torchinfo import summary
mc = MathorCup(2, 128, 2, cuda=True, num_layers=5, cuda_card=5)
summary(mc, ((128, 13, 2), (128, 4, 2)), batch_dim=None)

In [ ]:
def data_sample(data, lengths, index, offset=None):
    """
        使用前5个小时，一天前的4个小时，两天前的2个小时，一周前的2个小时进行预测
    """
    if offset == None:
        offset = [-1, -2, -3, -4, -5, -24, -25, -26, -27, -48, -49, -168, -169]
        offset.reverse()
    mini_offset = min(offset)
    selected = [index+item if index+item >= 0 else -1 for item in offset]
    pred = [index+item+1 if index+item >= 0 else -1 for item in offset]
#     print(pred[-1])
    data_samp = torch.stack([data[:, item, :] if item >=0 else torch.zeros_like(data[:, item, :]) for item in selected], dim=1)
    data_pred = torch.stack([data[:, item, :] if item >=0 else torch.zeros_like(data[:, item, :]) for item in pred], dim=1)
#     data_pred = data[:, [index], :]
#     return data_samp, data_pred
    return data_samp, data_pred
# data_samp, data_pred = data_sample(data_pad_T, data_lengths, 0)
# data_samp, data_pred = data_sample(data_pad, data_lengths, 16)
# data_samp.shape, data_pred.shape

In [ ]:
index, data_pad, lengths = make_data(read_train_data())
len(index), data_pad.shape, len(lengths)
# data_samp, data_pred = data_sample(data_pad, lengths, 2, )
# data_samp.shape, data_pred.shape
# mc(data_samp).shape

In [ ]:
def make_loader(data_pad, split_size, lengths, batch_size):
    total_num = data_pad.shape[1]
    train_num = int(total_num*split_size)
    val_num = total_num - train_num
    train_data_pad, train_lengths = data_pad[:, :train_num, :], [train_num if item > train_num else item for item in lengths]
    val_data_pad, val_lengths = data_pad[:, train_num:, :], [item-train_num if item > train_num else 0 for item in lengths] # 可能出负数，但是一般不会
    torch_Dataset = torch.utils.data.TensorDataset(train_data_pad, val_data_pad)
    loader = torch.utils.data.DataLoader(dataset=torch_Dataset, batch_size=batch_size, shuffle=True)
    return loader

In [ ]:
def train(df, cuda=True, cuda_card=0, epoch=5, lr=0.01, clip_value=0.001, hidden_num=32, log_per=1000, eval_per=100, num_layers=1, split_size=0.6, batch_size=32):
    train_mode = False
    if not isinstance(df, pd.DataFrame):
        train_mode = True
    lr_thresh = lr * 0.01
    mc = MathorCup(2, hidden_num, 2, cuda=cuda, cuda_card=cuda_card, num_layers=num_layers)
#     mc = nn.DataParallel(mc, output_device=[8])
    optim = torch.optim.Adam(mc.parameters(), lr=lr)
#     scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, lr_lambda=[lambda epoch: 0.9*epoch])
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.99)
    loss_func = nn.MSELoss()
    
    if not train_mode:
        df = [df]
    
    counter = 0
    local_loss = []
    cum_loss = []
    for epo in range(epoch):
        for d in df:
            index, data_pad, lengths = make_data(d)
            # 创建dataloader
            loader = make_loader(data_pad, split_size, lengths, batch_size)
            for step, (batch_train, batch_val) in enumerate(loader):
                counter += 1
#                 print(counter)
    #             print(step, batch_train.shape, batch_val.shape)
                batch_val_inp = batch_val[:, :-1, :]
                batch_val_real = batch_val[:, 1:, :]
                out = mc(batch_train, batch_val_inp)
#                 print(out.mean(), batch_val_real.mean())
                loss = loss_func(out, batch_val_real)
                optim.zero_grad()
                loss.backward()
                # 剪裁
                torch.nn.utils.clip_grad_value_(mc.parameters(), clip_value)
                optim.step()
                local_loss.append(loss.item())
                if math.isnan(loss.item()):
                    print(f"nan Counter: {counter+1}, index: {out.isnan()}")
                    break
                if (counter+1) % log_per == 0:
                    print(f"Epoch:{epo}, Counter:{counter+1}, Local Loss:{np.mean(local_loss)}")
                    cum_loss.append(np.mean(local_loss))
                    local_loss = []
                if (counter+1) % eval_per == 0:
                    eval_out = mc.evaluate(batch_train, batch_val.shape[1])
                    eval_loss = loss_func(eval_out, batch_val)
                    print(f"Epoch:{epo}, Counter:{counter+1}, Eval Loss:{eval_loss.item()}")
    #         if lr > lr_thresh:
    #             scheduler.step()
            
            lr = optim.state_dict()['param_groups'][0]['lr']
            if (epo+1) % 50 == 0:
                print(lr)
                
    #         if (epo+1) % 200 == 0:
    #             optim = torch.optim.Adamax(mc.parameters(), lr = optim.state_dict()['param_groups'][0]['lr'])
    #             scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=[lambda epoch: 0.1*epoch])
    #         for index in range(data_pad.shape[1]):
    #             counter += 1
    #             data_samp, data_pred = data_sample(data_pad, lengths, index, )
    #             output = mc(data_samp)
    #             loss = loss_func(output, data_pred)
    #             optim.zero_grad()
    #             loss.backward()
    #             # 剪裁
    # #             torch.nn.utils.clip_grad_value_(mc.parameters(), clip_value)
    #             optim.step()
    #             local_loss.append(loss.item())
    #             cum_loss.append(loss.item())
    #             if math.isnan(loss.item()):
    #                 print(f"nan Counter: {counter+1}, index: {index}")
    #                 break
    #             if (counter+1) % log_per == 0:
    #                 print(f"Epoch:{epo}, Counter:{counter+1}, Local Loss:{np.mean(local_loss)}")
    #                 local_loss = []
    #         scheduler.step()
    return mc, cum_loss
# mc = train(demo_df)

In [ ]:
def test(model, df, split_size=0.6, batch_size=16):
    index, data_pad, lengths = make_data(df)
    loader = make_loader(data_pad, split_size, lengths, batch_size)
    plot_list = []
    max_test = 100
    for step, (batch_train, batch_val) in enumerate(loader):
#         batch_val_inp = batch_val[:, :-1, :]
#         batch_val_real = batch_val[:, 1:, :]
        out = model.evaluate(batch_train, batch_val.shape[1])
#         print(out.shape, batch_val_real.shape)
#         plot_list.append((out[-1, :, :], batch_val_real[-1, :, :]))
        plot_list.append((out[-1, :, :], batch_val[-1, :, :]))
#     for index in tqdm(range(max(data_pad.shape[1], max_test))):
#         data_samp, data_pred = data_sample(data_pad, lengths, index, )
#         output = model(data_samp)
#         plot_list.append((output[:, -1, :], data_pred[:, -1, :]))
    return plot_list

# plot_list = test(mc, t_demo_df)
# plot_list[0]

In [ ]:
def sample_plot(plot_list, sample_id=None, plot_range=None):
    if sample_id==None:
        sample_id = random.randint(0, 127)
    if plot_range==None:
        temp = random.randint(0, 300)
        plot_range = [temp, temp+100]
    print(sample_id)
    print(plot_range)
    new_plot = torch.stack([torch.stack(item) for item in plot_list])
    print(new_plot.shape)
    Upload_x = new_plot[-1, 0, :, 0].detach().numpy()
    Upload_y = new_plot[-1, 1, :, 0].detach().numpy()
    
    Download_x = new_plot[-1, 0, :, 1].detach().numpy()
    Download_y = new_plot[-1, 1, :, 1].detach().numpy()
    # new_plot_x.shape
    fig, ax = plt.subplots(2, 1, squeeze=False)
    ax[0][0].plot(Upload_x[plot_range[0]:plot_range[1]])
    ax[0][0].plot(Upload_y[plot_range[0]:plot_range[1]], color='red', linestyle="--")
    ax[0][0].legend(["Raw", "Pred"])
    ax[0][0].set_title(f"Upload(GB) Prediction Plot From {plot_range[0]} To {plot_range[1]} In Sample {sample_id}")
    ax[1][0].plot(Download_x[plot_range[0]:plot_range[1]])
    ax[1][0].plot(Download_y[plot_range[0]:plot_range[1]], color='red', linestyle="--")
    ax[1][0].legend(["Raw", "Pred"])
    ax[1][0].set_title(f"Download(GB) Prediction Plot From {plot_range[0]} To {plot_range[1]} In Sample {sample_id}")
    plt.tight_layout()
    plt.show()
    return fig
# new_plot_x.detach().numpy()
# fig = sample_plot(plot_list)


In [ ]:
def main(demo=True):
    print("Reading Test Data...")
    try:
        test_df = read_test_data(demo)
    except Exception as e:
        print(e)
        print("Start Sampling...(About 3 mins)")
        test_df = data_filter(sample_data())
        save_df(test_df, "test_demo.csv")
    print("Reading Train Data...")
    try:
        train_df = read_train_data(demo)
    except Exception as e:
        print(e)
        print("Start Sampling...(About 3 mins)")
        train_df = data_filter(sample_data())
        save_df(train_df, "train_demo.csv")
    print("Training...")
    mc, cum_loss = train(train_df, cuda=True, cuda_card=0, epoch=600, lr=0.01, clip_value=0.001, hidden_num=128, log_per=100, eval_per=1000, num_layers=5, batch_size=16)
    print("Testing")
    plot_list = None
    plot_list = test(mc, test_df)
    return plot_list, cum_loss, mc
plot_list, cum_loss, mc = main(demo=False)

In [ ]:
fig = sample_plot(plot_list)

In [ ]:
plt.plot(cum_loss)

In [ ]:
plt.plot(cum_loss)

In [ ]:
para = sum([np.prod(list(p.size())) for p in mc.parameters()])
str(para/1000/1000)+"MB"

In [ ]:
input_ = data_pad[:32, :660, :]
input_.requires_grad_(requires_grad=False)
print(input_.shape)
mods = list(mc.modules())
out_sizes = []
for i in range(1, len(mods)):
    m = mods[i]
    print(m)
    out = m(input_)
    out_sizes.append(np.array(out.size()))
    input_ = out
mods[1:]